In [1]:
from gurobipy import Model, quicksum, GRB, max_, GenExpr

In [2]:
DC = set(range(1,3))
Cust = DC.copy()
Route = {(i,j) for i in DC for j in Cust}
CT = {1,2}

T = 2
Period = list(range(1, T+1))

In [5]:
tp = {i:1 for i in DC}
delivery = {(i,j):2 for i,j in Route}
demand = {(k,t):10 for k in Cust for t in Period}
dc_run_cost = {(j,ct):100 * ct for j in DC for ct in CT}
dc_inv_cost = {j:1 for j in DC}
dc_ub = {ct:40 * ct for ct in CT}
dc_new_cost = {j:1000 for j in DC}
cust_inv_cost = {k:30 for k in Cust}

In [ ]:
dc_ub

In [ ]:
model = Model()

In [ ]:
x, y, dc_inv, cust_inv, dc_new  = {}, {}, {}, {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for j in DC:
    for ct in CT:
        y[j,ct,0] = model.addVar(vtype="B", name=f'y[{j},{ct},{0}]')
        for t in Period:
            y[j,ct,t] = model.addVar(vtype="B", name=f'y[{j},{ct},{t}]')
    
for j in DC:
    dc_inv[j,0] = model.addVar(vtype="C", ub=0, name=f'dc_inv[{j},{t}]')  
    for t in Period:
        dc_inv[j,t] = model.addVar(vtype="C", name=f'dc_inv[{j},{t}]')    

for k in Cust:
    cust_inv[k,0] = model.addVar(vtype="C", ub=0, name=f'cust_inv[{k},{t}]')  
    for t in Period:
        cust_inv[k,t] = model.addVar(vtype="C", name=f'cust_inv[{k},{t}]')

model.update()

In [ ]:
Cust_Demand_Cons, DC_Running_Cons, DC_Flow_Cons, Center_Type_Cons, DC_UB_Cons = {}, {}, {}, {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            + cust_inv[k,t-1]
            ==
            demand[k,t]
            + cust_inv[k,t]
        )

for j,k in DRoute:
    for t in Period:
        DC_Running_Cons[j,k,t] = model.addConstr(
            x[j,k,t]
            <=
            quicksum(demand[k,t_] for t_ in Period) * quicksum(y[j,ct,t] for ct in CT) 
        )

for j in DC:
    for t in Period:
        DC_Flow_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt)
            + dc_inv[j,t-1]
            ==
            quicksum(x[j,k,t] for k in Cust)
            + dc_inv[j,t]
        )

for j in DC:
    for t in Period:
        Center_Type_Cons[j,t] = model.addConstr(
            quicksum(y[j,ct,t] for ct in CT)
            <=
            1
        )

for j in DC:
    for t in Period:
        DC_UB_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt) + dc_inv[j,t-1]
            <=
            quicksum(dc_ub[ct] * y[j,ct,t] for ct in CT)
        )

for j in DC:
    for ct in CT:
        for t in Period:
            
        
model.update()

In [ ]:
model.setObjective(
    quicksum(tp_cost[i,j] * x[i,j,t] for i,j in TRoute for t in Period) +
    quicksum(del_cost[j,k] * x[j,k,t] for j,k in DRoute for t in Period) +
    quicksum(dc_run_cost[j,ct] * y[j,ct,t] for j in DC for ct in CT for t in Period) +
    quicksum(dc_inv_cost[j] * dc_inv[j,t] for j in DC for t in Period) +
    quicksum(cust_inv_cost[k] * cust_inv[k,t] for k in Cust for t in Period) +
    quicksum(max_(y[j,ct,t] - y[j,ct,t-1], 0) * dc_new_cost[j] for j in DC for ct in CT for t in Period)
    ,GRB.MINIMIZE
)
model.update()

In [ ]:
model.optimize()

In [ ]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

In [ ]:
from gurobipy import GRBGenExpr

In [ ]:
GenExpr(0);